In [1]:
import pandas as pd
import time
from web3 import Web3

# --- Setup web3 and contract (example) ---
# Replace with my actual provider URL and contract details
provider_url = "http://127.0.0.1:7545"
web3 = Web3(Web3.HTTPProvider(provider_url))

if web3.is_connected():
    print("✅ Connected to Ganache successfully!")
else:
    print("❌ Connection failed. Ensure Ganache is running.")


# Set my default account (replace with my account)
web3.eth.default_account = web3.eth.accounts[0]  # or use private key management

# Replace with my contract ABI and address
contract_abi = [
	{
		"anonymous": False,
		"inputs": [
			{
				"indexed": False,
				"internalType": "string",
				"name": "vehicleID",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "speed_kmph",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "string",
				"name": "battery_percent",
				"type": "string"
			},
			{
				"indexed": False,
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			}
		],
		"name": "DataStored",
		"type": "event"
	},
	{
		"inputs": [
			{
				"internalType": "string",
				"name": "_vehicleID",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_speed_kmph",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_battery_percent",
				"type": "string"
			}
		],
		"name": "storeData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getRecord",
		"outputs": [
			{
				"internalType": "string",
				"name": "vehicleID",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "latitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "longitude",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "speed_kmph",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "battery_percent",
				"type": "string"
			},
			{
				"internalType": "uint256",
				"name": "timestamp",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "getRecordCount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]  # my contract ABI here
contract_address = "0xd40660925A19c8E0C8cA8905E0E4b2658957536F"
contract = web3.eth.contract(address=contract_address, abi=contract_abi)

print(f"✅ Connected to Smart Contract at {contract_address}")

# --- Load my CSV data ---
df = pd.read_csv('vehicle_tracking_data.csv')

print("Columns in DataFrame:", df.columns)

# Optional: Check required columns and set defaults if missing
required_columns = ['vehicle_id', 'latitude', 'longitude', 'speed_kmph', 'battery_percent']

for col in required_columns:
    if col not in df.columns:
        print(f"Warning: Missing column '{col}' in data. Filling with default values.")
        if col in ['vehicle_id']:
            df[col] = 'unknown'
        else:
            df[col] = 0  # numeric default

# --- Function to send data to smart contract ---
def send_vehicle_data(row):
    try:
        txn = contract.functions.storeData(
            str(row["vehicle_id"]),
            str(row["latitude"]),
            str(row["longitude"]),
            str(row["speed_kmph"]),
            str(row["battery_percent"])
        ).transact({'from': web3.eth.default_account, 'gas': 3000000})

        receipt = web3.eth.wait_for_transaction_receipt(txn)
        print(f"✅ Stored for {row['vehicle_id']} | Txn Hash: {receipt.transactionHash.hex()}")
    except Exception as e:
        print(f"❌ Failed to store data for {row.get('vehicle_id', 'unknown')}: {e}")

# --- Loop through rows and send data ---
for _, row in df.iterrows():
    send_vehicle_data(row)
    time.sleep(1)  # delay to avoid spamming the network

total = contract.functions.getRecordCount().call()
print(f"📦 Total records stored in contract: {total}")


✅ Connected to Ganache successfully!
✅ Connected to Smart Contract at 0xd40660925A19c8E0C8cA8905E0E4b2658957536F
Columns in DataFrame: Index(['vehicle_id', 'timestamp', 'latitude', 'longitude', 'speed_kmph',
       'battery_percent'],
      dtype='object')
✅ Stored for 1 | Txn Hash: 890a8fd5bee40e0b9610b89d8d80124fadd3fb0f4dd626d3b67e21f6217862ae
✅ Stored for 2 | Txn Hash: dc9e5f566d80860d7fa5594538cab3a5a3bf3f4aab958d573fcc72703f46310d
✅ Stored for 3 | Txn Hash: 12deb637eec2c894d9b43164f91f18138911d0284ce9ec42d14c0e158af293e3
✅ Stored for 4 | Txn Hash: 9398ca3d28fbc2adcf5bed1f10f544c74e7fe4a3465a181f75e36f70ee6546e3
✅ Stored for 5 | Txn Hash: d6dbb7a64343b2060722d8bf0713679db030e5a2eae74fc28f970b9ffc571e34
✅ Stored for 6 | Txn Hash: 9949f21d7e498c096743a7647c94959f0b7ea80136456d5bd4d41600a806d3ec
✅ Stored for 7 | Txn Hash: ba451a41e2c9f3aa92e9a3c237470cc57331e66f99aac1085d6fd1e7e30fd6a9
✅ Stored for 8 | Txn Hash: b3b6869c1b1b6bb6436a5fc6ed33f54b1225a0d6ceebeca293ee1e8e621199bc
✅ Store

In [29]:
# Print all function names available in the contract
for func in contract.functions:
    print(func.fn_name)
    

getRecord
getRecordCount
storeData


In [31]:
# Get the total number of records from the smart contract
total_records = contract.functions.getRecordCount().call()

# Display the first 5 stored records
for i in range(min(5, total_records)):
    record = contract.functions.getRecord(i).call()
    print(f"\nRecord {i + 1}:")
    print(f"  Vehicle ID   : {record[0]}")
    print(f"  Location     : ({record[1]}, {record[2]})")
    print(f"  Speed (km/h) : {record[3]}")
    print(f"  Battery (%)  : {record[4]}")
    print(f"  Timestamp    : {record[5]}")



Record 1:
  Vehicle ID   : 1
  Location     : (14.599986, 120.984765)
  Speed (km/h) : 44.97
  Battery (%)  : 99.92
  Timestamp    : 1748852511

Record 2:
  Vehicle ID   : 2
  Location     : (14.600463, 120.985423)
  Speed (km/h) : 37.66
  Battery (%)  : 99.85
  Timestamp    : 1748852512

Record 3:
  Vehicle ID   : 3
  Location     : (14.601017, 120.985876)
  Speed (km/h) : 35.31
  Battery (%)  : 99.81
  Timestamp    : 1748852513

Record 4:
  Vehicle ID   : 4
  Location     : (14.601326, 120.986204)
  Speed (km/h) : 42.42
  Battery (%)  : 99.77
  Timestamp    : 1748852514

Record 5:
  Vehicle ID   : 5
  Location     : (14.601857, 120.986613)
  Speed (km/h) : 29.87
  Battery (%)  : 99.75
  Timestamp    : 1748852515


In [33]:
# Get the total number of records from the smart contract
total_records = contract.functions.getRecordCount().call()

# Display all stored records
for i in range(total_records):
    record = contract.functions.getRecord(i).call()
    print(f"\nRecord {i + 1}:")
    print(f"  Vehicle ID   : {record[0]}")
    print(f"  Location     : ({record[1]}, {record[2]})")
    print(f"  Speed (km/h) : {record[3]}")
    print(f"  Battery (%)  : {record[4]}")
    print(f"  Timestamp    : {record[5]}")


Record 1:
  Vehicle ID   : 1
  Location     : (14.599986, 120.984765)
  Speed (km/h) : 44.97
  Battery (%)  : 99.92
  Timestamp    : 1748852511

Record 2:
  Vehicle ID   : 2
  Location     : (14.600463, 120.985423)
  Speed (km/h) : 37.66
  Battery (%)  : 99.85
  Timestamp    : 1748852512

Record 3:
  Vehicle ID   : 3
  Location     : (14.601017, 120.985876)
  Speed (km/h) : 35.31
  Battery (%)  : 99.81
  Timestamp    : 1748852513

Record 4:
  Vehicle ID   : 4
  Location     : (14.601326, 120.986204)
  Speed (km/h) : 42.42
  Battery (%)  : 99.77
  Timestamp    : 1748852514

Record 5:
  Vehicle ID   : 5
  Location     : (14.601857, 120.986613)
  Speed (km/h) : 29.87
  Battery (%)  : 99.75
  Timestamp    : 1748852515

Record 6:
  Vehicle ID   : 6
  Location     : (14.602335, 120.98712)
  Speed (km/h) : 54.66
  Battery (%)  : 99.73
  Timestamp    : 1748852516

Record 7:
  Vehicle ID   : 7
  Location     : (14.602846, 120.987505)
  Speed (km/h) : 34.56
  Battery (%)  : 99.67
  Timestamp   

In [21]:
!pip install pandas folium

In [35]:
from datetime import datetime
import pandas as pd

# Get the total number of records from the smart contract
total = contract.functions.getRecordCount().call()
print(f"📦 Total records stored in contract: {total}")

# Retrieve all records and store them in a list
records = []

for i in range(total):
    record = contract.functions.getRecord(i).call()
    records.append({
        "Vehicle ID": record[0],
        "Latitude": record[1],
        "Longitude": record[2],
        "Speed (km/h)": record[3],
        "Battery (%)": record[4],
        "Timestamp": datetime.fromtimestamp(record[5])  # convert UNIX to readable format
    })

# Create the DataFrame
df = pd.DataFrame(records)

# Optional: Preview the DataFrame
print(df.head())

📦 Total records stored in contract: 150
  Vehicle ID   Latitude   Longitude Speed (km/h) Battery (%)  \
0          1  14.599986  120.984765        44.97       99.92   
1          2  14.600463  120.985423        37.66       99.85   
2          3  14.601017  120.985876        35.31       99.81   
3          4  14.601326  120.986204        42.42       99.77   
4          5  14.601857  120.986613        29.87       99.75   

            Timestamp  
0 2025-06-02 16:21:51  
1 2025-06-02 16:21:52  
2 2025-06-02 16:21:53  
3 2025-06-02 16:21:54  
4 2025-06-02 16:21:55  


In [37]:
import folium

# Center the map on the first vehicle's location
start_coords = [df.loc[0, "Latitude"], df.loc[0, "Longitude"]]
vehicle_map = folium.Map(location=start_coords, zoom_start=16)

# Add vehicle markers
for _, row in df.iterrows():
    popup_info = (
        f"Vehicle ID: {row['Vehicle ID']}<br>"
        f"Speed: {row['Speed (km/h)']} km/h<br>"
        f"Battery: {row['Battery (%)']}%<br>"
        f"Timestamp: {row['Timestamp']}"
    )
    folium.Marker(
        location=[row["Latitude"], row["Longitude"]],
        popup=popup_info,
        tooltip=f"Vehicle {row['Vehicle ID']}"
    ).add_to(vehicle_map)

# Show or save map
vehicle_map  # If you're in Jupyter, this shows the map
# vehicle_map.save("vehicle_map.html")  # save to HTML

In [39]:
print(df.columns.tolist())


['Vehicle ID', 'Latitude', 'Longitude', 'Speed (km/h)', 'Battery (%)', 'Timestamp']


In [45]:
print(df.columns.tolist())

['Vehicle ID', 'Latitude', 'Longitude', 'Speed (km/h)', 'Battery (%)', 'Timestamp']


In [47]:
print(df.head(1).to_dict())

{'Vehicle ID': {0: '1'}, 'Latitude': {0: '14.599986'}, 'Longitude': {0: '120.984765'}, 'Speed (km/h)': {0: '44.97'}, 'Battery (%)': {0: '99.92'}, 'Timestamp': {0: Timestamp('2025-06-02 16:21:51')}}


In [55]:
import folium
import pandas as pd

data = {
    "vehicle_id": ["1"],
    "latitude": [14.599986],
    "longitude": [120.984765],
    "speed_kmph": [44.97],
    "battery_percent": [99.92],
    "timestamp": ["2025-06-02 16:21:51"]
}

df = pd.DataFrame(data)

vehicle_map = folium.Map(location=[df.loc[0, "latitude"], df.loc[0, "longitude"]], zoom_start=14)

for _, row in df.iterrows():
    popup_info = (
        f"Vehicle ID: {row['vehicle_id']}<br>"
        f"Speed: {row['speed_kmph']} km/h<br>"
        f"Battery: {row['battery_percent']}%<br>"
        f"Timestamp: {row['timestamp']}"
    )
    folium.Marker(
        location=[row["latitude"], row["longitude"]],
        popup=popup_info,
        tooltip=f"Vehicle {row['vehicle_id']}"
    ).add_to(vehicle_map)

# Show in Jupyter
vehicle_map

# Or save and open in browser
vehicle_map.save("vehicle_map.html")
import webbrowser
webbrowser.open("vehicle_map.html")


True

In [63]:
# Step 1: Import necessary libraries
import folium
import random

# Step 2: Generate simulated dataset of 150 vehicle records
data = []
base_lat, base_lon = 14.6390, 121.0330  # Approximate base location (Metro Manila area)

for vehicle_id in range(1, 150):
    lat = base_lat + random.uniform(-0.01, 0.01)
    lon = base_lon + random.uniform(-0.01, 0.01)
    speed = round(random.uniform(0, 60), 1)
    battery = round(random.uniform(20, 100), 2)
    data.append({
        "vehicle_id": vehicle_id,
        "lat": lat,
        "lon": lon,
        "speed": speed,
        "battery": battery
    })

# Step 3: Create the folium map centered around the average location
center_lat = sum(item['lat'] for item in data) / len(data)
center_lon = sum(item['lon'] for item in data) / len(data)
m = folium.Map(location=[center_lat, center_lon], zoom_start=14)

# Step 4: Add markers for each vehicle
for vehicle in data:
    popup_info = (
        f"Vehicle ID: {vehicle['vehicle_id']}<br>"
        f"Speed: {vehicle['speed']} km/h<br>"
        f"Battery: {vehicle['battery']}%"
    )
    folium.Marker(
        location=[vehicle['lat'], vehicle['lon']],
        popup=popup_info,
        icon=folium.Icon(color='blue', icon='car', prefix='fa')
    ).add_to(m)

# Step 5: Display the map in the notebook
m

# Step 6: Save the map as an HTML file - After running the code, a file named vehicle_map.html will be saved in the directory.
m.save("vehicle_map.html")


In [65]:
from web3 import Web3

ganache_url = "http://127.0.0.1:7545"  # or whatever your Ganache RPC is
w3 = Web3(Web3.HTTPProvider(ganache_url))

block = w3.eth.get_block(1547)  # or the latest block number from Remix
timestamp = block['timestamp']

from datetime import datetime
print("Timestamp:", timestamp)
print("Readable Time:", datetime.fromtimestamp(timestamp))


Timestamp: 1748852485
Readable Time: 2025-06-02 16:21:25


In [73]:
record = contract.functions.getRecord(0).call()

vehicle_id = record[0]
latitude = record[1]
longitude = record[2]
speed = record[3]
battery = record[4]
timestamp = record[5]

# Format Data Value
data_value = f"Vehicle ID: {vehicle_id}, Latitude: {latitude}, Longitude: {longitude}, Speed: {speed} km/h, Battery: {battery}%"
print(data_value)


Vehicle ID: 1, Latitude: 14.599986, Longitude: 120.984765, Speed: 44.97 km/h, Battery: 99.92%


In [75]:
from datetime import datetime

readable_time = datetime.fromtimestamp(timestamp)
print("Timestamp:", readable_time)


Timestamp: 2025-06-02 16:21:51


In [77]:
total_records = contract.functions.getRecordCount().call()
print("🔹 Total records now stored on blockchain:", total_records)


🔹 Total records now stored on blockchain: 150


In [79]:
record = contract.functions.getRecord(0).call()

vehicle_id = record[0]
latitude = record[1]
longitude = record[2]
speed = record[3]
battery = record[4]
timestamp = record[5]

from datetime import datetime
readable_timestamp = datetime.fromtimestamp(timestamp)

print("🔹 First stored record retrieved from blockchain:")
print("Timestamp:", readable_timestamp)
print("Device ID:", vehicle_id)
print("Data Type: Vehicle Telemetry")  # static since it contains multiple types
print(f"Data Value: Latitude={latitude}, Longitude={longitude}, Speed={speed} km/h, Battery={battery}%")


🔹 First stored record retrieved from blockchain:
Timestamp: 2025-06-02 16:21:51
Device ID: 1
Data Type: Vehicle Telemetry
Data Value: Latitude=14.599986, Longitude=120.984765, Speed=44.97 km/h, Battery=99.92%
